<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/simpsons_V2_newpix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/GitGo/datasets/garbage

In [ ]:

######################################################
from google.colab import drive
drive.mount('/content/drive')
import os 

!curl https://raw.githubusercontent.com/bxck75/colab_starter/main/setup.py -o /content/setup.py
!python /content/setup.py
!pip install tensorflow tensorflow-addons

######################################################

In [ ]:
os.chdir('/content/GitGo/datasets')
!unzip /content/drive/MyDrive/garbage.zip

In [ ]:
import glob,cv2
import numpy as np
from PIL import Image

!mkdir -p /content/GitGo/datasets/garbage_imgs_L
!mkdir -p /content/GitGo/datasets/garbage_imgs_P

i=0
for _f in sorted(glob.glob('/content/GitGo/datasets/garbage/*/*.jpg')):
  im = cv2.imread(_f)
  
  # You may need to convert the color.
  color_converted = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  pil_image=Image.fromarray(color_converted)
  
  h, w, c = im.shape
  split_fn = _f.split('/')
  if w < h:
    # portrait folder and rename to png
    pil_image.save('/content/GitGo/datasets/garbage_imgs_P/'+ split_fn[-1].replace('.jpg','.png'))
    i+=1
  else:
    # landscape folder and rename to png
    pil_image.save('/content/GitGo/datasets/garbage_imgs_L/'+ split_fn[-1].replace('.jpg','.png'))
    i+=1


In [ ]:
!rm -r /content/GitGo/datasets/garbage_imgs_P/*
!rm -r /content/GitGo/datasets/garbage_imgs_L/*

In [ ]:
#!rm -r /content/GitGo/datasets/garbage_imgs_P/*intro.jpg
#!rm -r /content/GitGo/datasets/garbage_imgs_P/*_ch.jpg

In [ ]:
import os
# Function to rename multiple files
def rename():
  i = 0
  path="/content/GitGo/datasets/garbage_imgs/"
  for _filename in sorted(os.listdir(path)):
    print(_filename)
    my_source = path + _filename
    my_dest = my_source.replace('.jpg','.png')
    # rename() function will
    # rename all the files
    os.rename(my_source, my_dest)
    i += 1


In [ ]:
#!rm -r /tmp/preprocess/*

In [ ]:
import os
os.chdir('/content/GitGo/piss-ant-pix2pix')
input_images = '/content/GitGo/datasets/garbage_imgs_P'
# resize
!python tools/process_custom.py --size 256 --pad --input_dir {input_images} --operation resize --output_dir /tmp/preprocess/_resized
# blank clones
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --operation blank --output_dir /tmp/preprocess/_blank
# edge clones
!mkdir -p /tmp/preprocess/_edge
!python tools/edge_custom.py --input_dir /tmp/preprocess/_resized --output_dir /tmp/preprocess/_edge
#!python tools/process_custom.py --input_dir /tmp/preprocess/_edge --operation blank --output_dir /tmp/preprocess/_blank_edge
# combine
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank --operation combine --output_dir  /tmp/preprocess/_resized/_combined_blank
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_edge --operation combine --output_dir  /tmp/preprocess/_resized/_combined_edge
#!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank_edge --operation combine --output_dir  /tmp/preprocess/_resized/_combined_blank_edge
# split
!python tools/split.py --dir /tmp/preprocess/_resized/_combined_blank
#!python tools/split.py --dir /tmp/preprocess/_resized/_combined_edge
#!python tools/split.py --dir /tmp/preprocess/_resized/_combined_blank_edge
# make outdirs
!mkdir -p /content/GitGo/datasets/garbage_blank_dataset
!mkdir -p /content/GitGo/datasets/simpsons_edge_dataset
#!mkdir -p /content/GitGo/datasets/simpsons_blank_edge_dataset
# copy to final folders
!cp -rf /tmp/preprocess/_resized/_combined_blank/ /content/GitGo/datasets/garbage_blank_dataset
!cp -rf /tmp/preprocess/_resized/_combined_edge/ /content/GitGo/datasets/simpsons_edge_dataset
#!cp -rf /tmp/preprocess/_resized/_combined_blank_edge/ /content/GitGo/datasets/simpsons_blank_edge_dataset

In [ ]:
!python /content/GitGo/piss-ant-pix2pix/tools/edge_custom.py --input_dir /tmp/preprocess/_resized --output_dir /tmp/preprocess/_edges

In [ ]:
%cd /content/GitGo/piss-ant-pix2pix
import cv2,glob
from google.colab.patches import cv2_imshow
!mkdir -p '/tmp/preprocess/_edges'
def edge(_in,_out):
    image = cv2.imread(_in)
    #cv2_imshow(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # show the original and blurred images
    #cv2_imshow(image)
    #cv2_imshow( blurred)
    # compute a "wide", "mid-range", and "tight" threshold for the edges
    # using the Canny edge detector
    #wide = cv2.Canny(blurred, 10, 200)
    mid = cv2.Canny(blurred, 30, 150)
    #tight = cv2.Canny(blurred, 240, 250)
    # show the output Canny edge maps
    #cv2_imshow(~wide)
    #cv2_imshow(~mid)
    #cv2_imshow(~tight)
    cv2.imwrite(_out, ~mid)


for img_path in glob.glob('/tmp/preprocess/_resized/*.png'):
    print(img_path)
    new_img_path = img_path.replace('_resized','_edges')
    print(new_img_path)
    edge(img_path, new_img_path)


!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_edges --operation combine --output_dir  /tmp/preprocess/_combined_edge
!python tools/split.py --dir /tmp/preprocess/_combined_edge
!mkdir -p /content/GitGo/datasets/garbage_edge_dataset
!cp -rf /tmp/preprocess/_combined_edge/ /content/GitGo/datasets/garbage_edge_dataset

In [ ]:
print(os.path.exists('/tmp/preprocess/_edges'))
!ls /tmp/preprocess/_edges

In [ ]:
print(len( glob.glob('/tmp/preprocess/_edges/*.png')))
edges_imgs = glob.glob('/tmp/preprocess/_edges/*.png')

In [ ]:
!mkdir -p  /tmp/preprocess/_blank_edge
!python tools/process_custom.py --input_dir /tmp/preprocess/_edges --operation blank --output_dir /tmp/preprocess/_blank_edge
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank_edge --operation combine --output_dir  /tmp/preprocess/_combined_blank_edge
!python tools/split.py --dir /tmp/preprocess/_combined_blank_edge
!mkdir -p /content/GitGo/datasets/garbage_blank_edge_dataset
!cp -rf /tmp/preprocess/_combined_blank_edge /content/GitGo/datasets/garbage_blank_edge_dataset

In [ ]:
import glob
files = glob.glob('/content/GitGo/datasets/garbage_edge_dataset/_combined_edge/train/*.png')
print(len(files))

In [ ]:
!rm -r /content/drive/MyDrive/garbage_metrics/*

In [ ]:
!pip install -U tensorboard-plugin-profile
%reload_ext tensorboard
logs_base_dir = '/content/garbage_metrics'
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

In [ ]:
import os,warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore')
os.chdir('/content/GitGo/piss-ant-pix2pix')
custom_dataset = '/content/GitGo/datasets/garbage_edge_dataset/_combined_edge/train'
train_metrics = '/content/garbage_metrics'
!mkdir -p {train_metrics}

# CHECK IF ITS A RETRAINING
if os.path.exists('/content/simpsons_metrics/checkpoint'):
    retrain = '--checkpoint '+ train_metrics
else:
    retrain = ''
# train
!python pix2pix_custom.py \
    --summary_freq 50 \
    --ngf 64 \
    --ndf 64 \
    --save_freq 50 \
    --display_freq 10 \
    --progress_freq 50 \
    --batch_size 64 \
    --mode train \
    --output_dir {train_metrics} \
    --max_epochs 3 \
    --input_dir {custom_dataset} \
    {retrain} \
    --which_direction BtoA

In [ ]:
!rm -r /content/garbage_metrics/images

In [ ]:
!zip -r /content/drive/MyDrive/garbage_metrics_backup.zip /content/garbage_metrics